In [73]:
# Logistic Regression & Decision Trees on a Kaggle Dataset
# Dataset Description
# We will use the Pima Indians Diabetes Dataset (available on Kaggle: Diabetes Dataset).

# Target Variable: Outcome

# 1 → Patient has diabetes

# 0 → Patient does not have diabetes

# Features:

# Pregnancies

# Glucose

# BloodPressure

# SkinThickness

# Insulin

# BMI

# DiabetesPedigreeFunction

# Age

# Tasks
# Part A: Data Preprocessing
# Load the dataset into a Pandas DataFrame.

# Perform exploratory data analysis (EDA):

# Show the first 5 rows.

# Check for missing values.

# Display basic statistics (mean, std, etc.).

# Split the data into train (70%) and test (30%) sets.

# Part B: Logistic Regression
# Train a Logistic Regression model using the training data.

# Predict on the test set.

# Calculate and interpret the following metrics:

# Accuracy

# Precision, Recall, F1-score

# Confusion Matrix

# ROC Curve & AUC Score

# Guiding Question:

# Why is AUC-ROC a better evaluation metric than just accuracy in imbalanced datasets?
# Part C: Decision Tree Classifier
# Train a Decision Tree Classifier (use max_depth = 4).

# Predict on the test set.

# Calculate the same metrics as Logistic Regression.

# Plot the Decision Tree and interpret the first split (which feature did it choose and why?).

# Guiding Question:

# Compare the Decision Tree’s interpretability vs. Logistic Regression’s coefficients.

# Which one would you prefer in a medical diagnosis scenario? Why?

# Part D: Comparison & Reflection
# Compare Logistic Regression and Decision Tree results (accuracy, precision, recall, F1, AUC).

# Discuss:

# Which model performs better?

# Which model is easier to interpret for doctors?

# If the dataset is slightly imbalanced (more 0’s than 1’s), which metric (Precision or Recall) should be prioritized in diagnosing diabetes? Why?

In [74]:
pip install pandas numpy matplotlib seaborn scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import tree



In [76]:
#Load Dataset 
df = pd.read_csv('diabetes.csv')

In [77]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [78]:
#Dataset quick peek - PART A
df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [79]:
#Show first 5 rows
df.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [80]:
#Check for missing values 
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [81]:
#display basic statistics 
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [82]:
#Data cleaning 
#fill missing columns with media 
df.fillna(df.median(numeric_only=True), inplace=True)
#remove duplicates
df.duplicated().sum()
df = df.drop_duplicates()


In [83]:
##Feature Engineering 

# Seperate feature and target 
X = df.drop('Outcome', axis=1)
y = df['Outcome']

##Onehot encoding 

#Identify categorical features 
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

#Onehot encoding categorical features 
if len(categorical_features) > 0:
    onehot_encoder = OneHotEncoder(sparse_output=False)
    X_categorical = onehot_encoder.fit_transform(X[categorical_features])
    X_categorical = pd.DataFrame(X_categorical, columns=onehot_encoder.get_feature_names_out(categorical_features))
else:
    X_categorical = pd.DataFrame(columns=[])

#combine numerical and categorical features 
X_all = pd.concat([X[numerical_features], X_categorical], axis=1)


In [84]:
#Feature Scaling 
scaler = StandardScaler()
X_all = scaler.fit_transform(X_all)
y = y.values.reshape(-1, 1)

In [ ]:
#Split data into train and test 
X_train, X_test, y_train, y_test = train_test_split(X_all, y,train_size=0.7, test_size=0.3, random_state=42,stratify=y)

In [86]:
#Model 1: Logistic Regression 
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

#Evaluate Model 1 
accuracy_lr = accuracy_score(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr)
recall_lr = recall_score(y_test, y_pred_lr)
f1_lr = f1_score(y_test, y_pred_lr)
confusion_matrix_lr = confusion_matrix(y_test, y_pred_lr)
roc_auc_lr = roc_auc_score(y_test, y_pred_lr)

print("Logistic Regression Metrics:")
print(f"Accuracy: {accuracy_lr}")
print(f"Precision: {precision_lr}")
print(f"Recall: {recall_lr}")
print(f"F1-score: {f1_lr}")
print(f"Confusion Matrix:\n {confusion_matrix_lr}")
print(f"ROC AUC Score: {roc_auc_lr}")

Logistic Regression Metrics:
Accuracy: 0.7359307359307359
Precision: 0.6172839506172839
Recall: 0.625
F1-score: 0.6211180124223602
Confusion Matrix:
 [[120  31]
 [ 30  50]]
ROC AUC Score: 0.7098509933774835


t:\ml_model_eval\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [87]:
#Model 2: Decision Tree 
dt = DecisionTreeClassifier(max_depth=4)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

#Evaluate Model 2 
accuracy_dt = accuracy_score(y_test, y_pred_dt)
precision_dt = precision_score(y_test, y_pred_dt)
recall_dt = recall_score(y_test, y_pred_dt)
f1_dt = f1_score(y_test, y_pred_dt)
confusion_matrix_dt = confusion_matrix(y_test, y_pred_dt)
roc_auc_dt = roc_auc_score(y_test, y_pred_dt)

print("Decision Tree Metrics:")
print(f"Accuracy: {accuracy_dt}")
print(f"Precision: {precision_dt}")
print(f"Recall: {recall_dt}")
print(f"F1-score: {f1_dt}")
print(f"Confusion Matrix:\n {confusion_matrix_dt}")
print(f"ROC AUC Score: {roc_auc_dt}")

Decision Tree Metrics:
Accuracy: 0.70995670995671
Precision: 0.5585585585585585
Recall: 0.775
F1-score: 0.6492146596858639
Confusion Matrix:
 [[102  49]
 [ 18  62]]
ROC AUC Score: 0.7252483443708609
